In [ ]:
# default_exp chem

# Chem

> Core comp chem functions implemented with RDKit

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from mrl.core import *
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import rdMMPA
from rdkit.Chem.FilterCatalog import *

//anaconda3/envs/insight/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [ ]:
# export
def to_mol(smile_or_mol):
    if type(smile_or_mol) == str:
        mol = Chem.MolFromSmiles(smile_or_mol)
    else:
        mol = smile_or_mol
        
    return mol

def to_smile(smile_or_mol):
    if type(smile_or_mol) == str:
        smile = smile_or_mol
    else:
        smile = Chem.MolToSmiles(smile_or_mol)
        
    return smile